# Imports

In [4]:
import matplotlib
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib as mpl
import seaborn as sns; sns.set()

from sqlalchemy import create_engine

%matplotlib inline
pd.set_option('display.max_columns', None)


In [7]:
# Load credentials from json file
with open("credentials.json", "r") as file:  
    creds = json.load(file)

USERNAME = creds['USERNAME']
PASSWORD = creds['PASSWORD']

# Connect to the db

In [13]:
#create the engine and connection
engine = create_engine('postgresql+psycopg2:'\
                       USERNAME\
                       PASSWORD\
                       '@49.12.227.17:5432/'\
                       'nba_games')

print('Database session created')
con = engine.connect()

Database session created


# Birthplace
## Find all allstar MVP players

In [3]:
df = pd.read_sql_query("""
SELECT allstar_mvps.player_id, players.name
FROM allstar_mvps LEFT JOIN players
ON allstar_mvps.player_id=players.id
""", con)
df

,player_id,name
0,129,Paul Arizin
1,124,Tiny Archibald
2,208,Charles Barkley
3,234,Rick Barry
4,259,Elgin Baylor
...,...,...
68,4391,Dwyane Wade
69,4522,Jerry West
70,4527,Russell Westbrook
71,4527,Russell Westbrook


## Find all players born is in New York

In [4]:
df = pd.read_sql_query("""
SELECT players.name
FROM players
WHERE birth_place LIKE %(c)s
ORDER BY name
""", con, params={'c': '%New York%'})
df


,name
0,Adonis Jordan
1,A.J. Wynder
2,Albert King
3,Al Bianchi
4,Al Butler
...,...
398,York Larese
399,Zaid Abdul-Aziz
400,Zeke Sinicola
401,Zeke Zawoluk


## Which players born in New York hold a MVP title?

In [5]:
df = pd.read_sql_query("""
SELECT DISTINCT ny.name, ny.id
FROM (SELECT players.name as name, players.id as id
FROM players
WHERE birth_place LIKE %(c)s) as ny 
INNER JOIN allstar_mvps
ON ny.id=allstar_mvps.player_id
""", con, params={'c': '%New York%'})
df


,name,id
0,Bob Cousy,865
1,Bob Lanier,2454
2,Julius Erving,1241
3,Lenny Wilkens,4578
4,Michael Jordan,2238
5,Randy Smith,3980
6,Tiny Archibald,124


# Which players have the same height?

In [6]:
df = pd.read_sql_query("""
SELECT A.name AS name_A, B.name AS players_B, A.height
FROM players as A, players as B
WHERE A.id <> B.id
AND A.height = B.height
ORDER BY A.height DESC
""", con)
df

,name_a,players_b,height
0,Manute Bol,Gheorghe Mureșan,7-7
1,Gheorghe Mureșan,Manute Bol,7-7
2,Shawn Bradley,Yao Ming,7-6
3,Yao Ming,Shawn Bradley,7-6
4,Sim Bhullar,Tacko Fall,7-5
...,...,...,...
1778389,Chet Aubuchon,Andre Barrett,5-10
1778390,Chet Aubuchon,Tyson Wheeler,5-10
1778391,Chet Aubuchon,Khalid El-Amin,5-10
1778392,Chet Aubuchon,Pierre Jackson,5-10


# List the top 10 of the players having the most Player of the Month Awards

In [7]:
df=pd.read_sql_query("""
select  COUNT(monthly_player_awards.season), MAX(players.full_name), monthly_player_awards.player_id
from monthly_player_awards 
INNER JOIN players on monthly_player_awards.player_id=players.id 
Group By player_id
ORDER BY count DESC
LIMIT 10
""", con)

df

,count,max,player_id
0,45,LeBron Raymone James,2065
1,19,Kevin Wayne Durant,1148
2,19,Michael Jeffrey Jordan,2238
3,16,Kobe Bean Bryant,564
4,16,Shaquille Rashaun O'Neal,3184
5,14,Christopher Emmanuel Paul,3309
6,12,Carmelo Kyam Anthony,115
7,10,Russell Westbrook III,4527
8,10,David Maurice Robinson,3618
9,10,Wardell Stephen Curry II,921


# Using table teams, create a new column that writes the messages "More than 2000 wins", "2000 wins" or "Less than 2000 wins"  depending the nr of wins it has, and also print the name and the number of wins for each team.


In [8]:
df = pd.read_sql_query("""
SELECT team_name, total_wins,
CASE
    WHEN total_wins > 2000 THEN 'More than 2000 wins'
    WHEN total_wins = 2000 THEN '2000 wins'
    ELSE 'Less than 2000 wins'
END AS WinsCategory
FROM teams
""", con)
df

,team_name,total_wins,winscategory
0,Atlanta Hawks,2762,More than 2000 wins
1,Boston Celtics,3416,More than 2000 wins
2,Brooklyn Nets,1849,Less than 2000 wins
3,Charlotte Hornets,1046,Less than 2000 wins
4,Chicago Bulls,2224,More than 2000 wins
5,Cleveland Cavaliers,1862,Less than 2000 wins
6,Dallas Mavericks,1605,Less than 2000 wins
7,Denver Nuggets,2154,More than 2000 wins
8,Detroit Pistons,2752,More than 2000 wins
9,Golden State Warriors,2784,More than 2000 wins


# Views
## Create a view with the team id, team name and number of championships

In [ ]:
df = pd.read_sql_query("""
CREATE VIEW Wins AS
SELECT teams.team_id, teams.team_name, most_wins.count
FROM teams INNER JOIN
(Select champions.team_id, count(champions.team_id)
FROM champions
Group By team_id) as most_wins
ON teams.team_id=most_wins.team_id
ORDER BY count DESC;

Select * from Wins
""", con)
df

,team_id,team_name,count
0,2,Boston Celtics,17
1,14,Los Angeles Lakers,15
2,5,Chicago Bulls,6
3,10,Golden State Warriors,5
4,27,San Antonio Spurs,5
5,23,Philadelphia 76ers,3
6,16,Miami Heat,3
7,9,Detroit Pistons,3
8,11,Houston Rockets,2
9,20,New York Knicks,2


## Print from the view all the teams from Miami or Milwaukee that have a championship

In [ ]:

df = pd.read_sql_query("""
SELECT *
FROM Wins
Where team_name like %(c1)s OR team_name like %(c2)s
""", con, params={'c1': 'Miami%', 'c2': 'Mil%'})
df


,team_id,team_name,count
0,16,Miami Heat,3
1,17,Milwaukee Bucks,1


## Update view with total wins and total loses


In [ ]:
df = pd.read_sql_query("""
CREATE OR REPLACE VIEW Wins AS
SELECT teams.team_id, teams.team_name, most_wins.count, teams.total_wins,teams.total_losses
FROM teams INNER JOIN
(Select champions.team_id, count(champions.team_id)
FROM champions
Group By team_id) as most_wins
ON teams.team_id=most_wins.team_id
ORDER BY count DESC;

Select * from Wins
""", con)
df

,team_id,team_name,count,total_wins,total_losses
0,2,Boston Celtics,17,3416,2362
1,14,Los Angeles Lakers,15,3374,2294
2,5,Chicago Bulls,6,2224,2129
3,10,Golden State Warriors,5,2784,2992
4,27,San Antonio Spurs,5,2563,1713
5,23,Philadelphia 76ers,3,2892,2714
6,16,Miami Heat,3,1329,1220
7,9,Detroit Pistons,3,2752,2920
8,11,Houston Rockets,2,2260,2011
9,20,New York Knicks,2,2795,2981


# Find all players that have both awards and other awards and print the name of the award as well.


In [ ]:
df = pd.read_sql_query("""
SELECT players.id, name, height, player_awards.award, other_player_awards.award
FROM players
INNER JOIN player_awards ON players.id=player_awards.player_id
INNER JOIN other_player_awards ON players.id=other_player_awards.payer_id
""", con)
df

,id,name,height,award,award
0,952,Bob Davies,6-1,All-League 1st Team,SLAM 500 Greatest NBA Players of All Time
1,952,Bob Davies,6-1,All-League 1st Team,NBA Silver Anniversary Team
2,2893,George Mikan,6-10,All-League 1st Team,SLAM 500 Greatest NBA Players of All Time
3,2893,George Mikan,6-10,All-League 1st Team,Bill Simmons Hall of Fame Pyramid
4,2893,George Mikan,6-10,All-League 1st Team,50 Greatest Players
...,...,...,...,...,...
5339,2065,LeBron James,6-9,All-League 3rd Team,Sporting News MVP
5340,2065,LeBron James,6-9,All-League 3rd Team,Sporting News MVP
5341,2065,LeBron James,6-9,All-League 3rd Team,Sporting News MVP
5342,2065,LeBron James,6-9,All-League 3rd Team,Sporting News Rookie of the Year
